# Kreiranje GUI na podlagi opisov

## Uvoz knjižnic in nastavitev okolja

In [39]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI #before: langchain.chat_models
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
import os
from langchain.chains import SequentialChain
import time


load_dotenv()
handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [40]:
MODEL = "gemini-2.5-flash" # before: gpt-4-1106-preview

OUTPUT_FOLDER = "generated_guis"
DATASET = "dataset.csv"

df = pd.read_csv(DATASET, header=0, sep=";")

## Definicija LLM verig za generiranje GUI kode

### 1. Instruction prompt

In [41]:
from prompts import SUMMARY2CODE_ZS

def instruction(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=SUMMARY2CODE_ZS, callbacks=[handler])

    try:
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)

    return(response)

### 2. Prompt decomposition (zero-shots)

In [42]:
from prompts import SUMMARY2REQUIREMENTS_ZS, REQUIREMENTS2IMPLEMENTATION_ZS, IMPLEMETATION2STRUCTURE_ZS, STRUCTURE2CODE_ZS

def pd_zs(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIREMENTS_ZS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_ZS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_ZS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])

    return(seq_chain.run(summary, callbacks=[handler]))


### 3. Prompt decomposition (few-shots)


In [43]:
from prompts import SUMMARY2REQUIRMENTS_FS, REQUIREMENTS2IMPLEMENTATION_FS, IMPLEMETATION2STRUCTURE_FS, STRUCTURE2CODE_FS

def pd_fs(summary, temp=0.7):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIRMENTS_FS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_FS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_FS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_FS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])

    return(seq_chain.run(summary, callbacks=[handler]))

### 4. Refined instruction prompt

In [44]:
from prompts import REFINE_SUMMARY_ZS

def ref_summary(summary, temp=1):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=REFINE_SUMMARY_ZS, callbacks=[handler])

    try:
        response = chain.run(summary=summary, callbacks=[handler])

    except Exception as e:
        print(e)

    return(response)

In [45]:
from prompts import REF_SUMMARY2CODE_ZS

def ref_instruction(summary, temp=1):
    llm = ChatGoogleGenerativeAI(model=MODEL,temperature=temp)
    chain = LLMChain(llm=llm, prompt=REF_SUMMARY2CODE_ZS, callbacks=[handler])

    try:
        refined_summary = ref_summary(summary)
        response = chain.run(summary=refined_summary, callbacks=[handler])
    except Exception as e:
        print(e)

    return(response)

## Izvleček HTML kode in shranjevanje datotek

In [46]:
import re

def extract_html(code, file, folder):
    html_content = re.search(r'<!DOCTYPE html>.*</html>', code, re.DOTALL)
    extracted_text = html_content.group()
    try:
        with open(f"{folder}/{file}.html", 'w', encoding="utf-8") as f:
            f.write(extracted_text)
    except Exception as e:
        print(f"{file} failed")
        print(e)

## Glavna funkcija za prototipiranje


In [47]:
def prototyping(summary, folder):
    timings = {}

    start = time.time()
    code = instruction(summary)
    extract_html(code, "instruction", folder)
    timings["instruction"] = time.time() - start

    start = time.time()
    code = pd_zs(summary)
    extract_html(code, "pd_zs", folder)
    timings["pd_zs"] = time.time() - start

    start = time.time()
    code = pd_fs(summary)
    extract_html(code, "pd_fs", folder)
    timings["pd_fs"] = time.time() - start

    start = time.time()
    code = ref_instruction(summary)
    extract_html(code, "ref_instruction", folder)
    timings["ref_instruction"] = time.time() - start

    return timings


## Zagon pipeline-a za generiranje rezultatov


In [48]:
def pipeline():
    all_timings = []
    for index, row in df.iterrows():
        path = f"{OUTPUT_FOLDER}/{row['UI_Number']}"

        if os.path.exists(path):
            continue

        os.makedirs(path)
        timings = prototyping(row['Summary'], path)
        timings_record = {
            "UI_Number": row['UI_Number'],
            **timings
        }
        all_timings.append(timings_record)

    timings_df = pd.DataFrame(all_timings)
    output_csv_path = "generation_timings.csv"

    if os.path.exists(output_csv_path):
        existing_df = pd.read_csv(output_csv_path, encoding="utf-8")
        combined_df = pd.concat([existing_df, timings_df], ignore_index=True)
        combined_df.to_csv(output_csv_path, index=False, encoding="utf-8")
    else:
        timings_df.to_csv(output_csv_path, index=False, encoding="utf-8")

    return timings_df


In [ ]:
pipeline()

### Testiranje in ročno izvajanje ter shranjevanje rezultatov

In [ ]:
def test_pipeline():
    for index, row in df.iterrows():
        code = ref_instruction(row['Summary'])
        file = "ref_instruction"
        folder = f"{OUTPUT_FOLDER}/{row['UI Number']}"

        html_content = re.search(r'<!DOCTYPE html>.*</html>', code, re.DOTALL)
        extracted_text = html_content.group()
        try:
            with open(f"{folder}/{file}.html", 'w', encoding="utf-8") as f:
                f.write(extracted_text)
                print(row['UI Number'])
        except Exception as e:
            print(f"{file} failed")
            print(e)

test_pipeline()